In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy.fft import rfftfreq, next_fast_len
import matplotlib.pyplot as plt
import scipy.special as ss
import time
import math
from mkl_fft import rfft_numpy
from numba import njit, prange
import numba, mkl
import sys
import concurrent.futures

from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigs, eigsh
import mpl_toolkits.mplot3d as mplot3d
from tqdm import tqdm_notebook


# def interp(f_0,rparams,m):
#     m=m
#     f = np.zeros(np.shape(rparams.r))
#     sum_list = []
#     for i in range(1,N+1):
#         sum_term = 2*rparams.alpha_0[i]*ss.jv(0,2*np.pi*rparams.beta*rparams.r)
#         sum_term /= rparams.alpha_0[i]**2 - (2*np.pi*rparams.beta*rparams.r)**2
#         sum_term *= f_0[i]/ss.jv(1,rparams.alpha_0[i])
#         sum_list.append(sum_term)
#     f = sum(sum_list)
#     return f

def interpolation(f_0,rparams,m):
    m=m
    f = np.zeros((N+1,Nz))
    den1 = (rparams.alpha_0**2).reshape(1,1,-1).repeat(N+1,0).repeat(Nz,1)
    r_3d = rparams.r.reshape(rparams.r.shape +(1,)).repeat(N+1,-1)
    den = den1 - (2*np.pi*rparams.beta*r_3d)**2
    f = np.nan_to_num(2*rparams.alpha_0*f_0/ss.jv(1,rparams.alpha_0),posinf=0,neginf=0)
    f = np.nan_to_num(f/den,posinf=0,neginf=0)
    f = np.sum(f,axis=-1) *ss.jv(0,2*np.pi*rparams.beta*rparams.r)
    return f
    

    
class A(LinearOperator):
    def __init__(self,N,Nz,D,Udd,psi,m,mu,dtype='float32'):
            self.Nr = N+1
            self.Nz = Nz
            N_tot = self.Nr*self.Nz
            self.psi = psi
            self.shape = (N_tot,N_tot)
            self.dtype = np.dtype(dtype)
    def _matvec(self,f):
    
        f = np.reshape(f,(self.Nr,self.Nz)) ##reshapes from (Nr*Nz,1) to (Nr,Nz)
        laplacian = (k_mag**2)/2
        C = D*calc_dd_cont_cyl(np.abs(psi)**2,rparams,Udd,m) 
        f_k1 = piecewise_mult_2D(laplacian, DHFT(f, rparams,m))
        f_k1 = IDHFT(f_k1, rparams,m) ##applies kinetic operator
        f_X = D*calc_dd_cont_cyl(psi*f, rparams,Udd,m)*psi
        f = (V_trap + C -mu)*f + f_k1 +2*f_X
    
        f_k2 = piecewise_mult_2D(laplacian, DHFT(f, rparams,m))
        f_k2 = IDHFT(f_k2, rparams,m) ##applies kinetic operator
        
        f = f_k2 + (V_trap -mu + C)*f
        
        f = np.reshape(f,(self.Nr*self.Nz,1))  ##reshapes from (Nr,Nz) to (Nr*Nz,1)
        return f
    
    
    def _rmatvec(self,f): ### for just H0, rmatvec is the same as matvec, I think its true for the full A operator
        
        f = np.reshape(f,(self.Nr,self.Nz)) ##reshapes from (Nr*Nz,1) to (Nr,Nz)
        laplacian = (k_mag**2)/2
        C = D*calc_dd_cont_cyl(np.abs(psi)**2,rparams,Udd,m) 
        f_k1 = piecewise_mult_2D(laplacian, DHFT(f, rparams,m))
        f_k1 = IDHFT(f_k1, rparams,m) ##applies kinetic operator
        f_X = D*calc_dd_cont_cyl(psi*f, rparams,Udd,m)*psi
        f = (V_trap + C -mu)*f + f_k1 +2*f_X
        
        f_k2 = piecewise_mult_2D(laplacian, DHFT(f, rparams,m))
        f_k2 = IDHFT(f_k2, rparams,m) ##applies kinetic operator
        
        f = f_k2 + (V_trap -mu + C)*f
        
        f = np.reshape(f,(self.Nr*self.Nz,1))  ##reshapes from (Nr,Nz) to (Nr*Nz,1)
        return f
     
def BdG_lowest_excitation_energy_order_m(N,Nz,D,Udd,psi,m,mu,interp):
    if interp:
        psi = interpolation(psi,rparams,m)
    A_ = A(N,Nz,D,Udd,psi,m,mu)
    
    ###### energy scales with N,Nz when it shouldn't
    ######cant use sigma because it can't calculate inverse for interacting case I think
    evals_small, evecs_small = eigs(A_,3,which='SR',maxiter=5000,tol=1e-1)
    return evals_small, evecs_small
    
            
def init_threads(num_threads):
    """Controls the number of threads launched for MKL/Numba."""
    numba.config.THREADING_LAYER = 'omp'
    numba.set_num_threads(num_threads)
    mkl.set_num_threads(num_threads)


def get_S(N, alpha, k, J0, m):
    """S is based on method in report. ss is short-hand for scipy.special."""
    if m==0:
        S = ss.jv(0, alpha[k] * alpha / J0[N + 1]) ** 2
        S /= ss.jv(0, alpha) ** 2
        S = np.sqrt(1 + np.sum(S) - S[0])
        S /= np.abs(ss.jv(0, alpha[k])) / 2  # defining S based on method in report, see eq. 3.10
    else:
        S = alpha[-1]
    return S


def get_C(N, alpha, S, m):
    """C is an orthogonal Bessel Transformation Matrix."""
    if m==0:
        # see report Eq. 3.9 for the below
        C = np.fromfunction(lambda i, j_: ss.jv(0, alpha[i] * alpha[j_] / S), (N + 1, N + 1), dtype=int)
        const = np.fromfunction(lambda i, j_: S * np.abs(ss.jv(0, alpha[i]) * ss.jv(0, alpha[j_])) / 2, (N + 1, N + 1), dtype=int)
        C /= const
    else:
        # see report Eq. 3.9 for the below
        C = np.fromfunction(lambda i, j_: ss.jv(m, alpha[i] * alpha[j_] / S), (N + 1, N + 1), dtype=int)
        const = np.nan_to_num(np.fromfunction(lambda i, j_: S * np.abs(ss.jv(m+1, alpha[i]) * ss.jv(m+1, alpha[j_])) / 2, (N + 1, N + 1), dtype=int),posinf=0,neginf=0)
        C = np.nan_to_num(C/const,posinf=0,neginf=0)
    return C


def radialsetup(N, Nz, b, m):
    """Defines the grid in the r direction."""

    class rparams:
        k = int(N / 4)  # k is the optimal value for making C orthogonal
        # alpha is the array of the zeros of the 1st derivative of the 0th order Bessel function, including 0 at the start - i.e. N+1 points in total
        alpha_0 = np.concatenate(([0], ss.jnp_zeros(0, N)))
        if m ==0:
            alpha = alpha_0 
        else:
            alpha = np.concatenate(([0], ss.jn_zeros(m, N)))
        J0 = ss.jn_zeros(0, N + 2)  # zeros of 0th order Bessel function
        if m==0:
            J = np.abs(ss.j0(alpha))
        if m !=0:
            J = np.abs(ss.jv(m+1,alpha))
        J = J.reshape(-1, 1)  # reshaping to 2D
        S = get_S(N, alpha, k, J0, m)
        C = get_C(N, alpha, S, m)
        beta = S / (2 * np.pi * b)  # max rho value
        rho = alpha * beta / S  # this comes from the referenced paper in the report
        r = alpha * b / S
        
        # The below originates from Rob:
        dr = (np.roll(r, -1) - np.roll(r, 1)) / 2  # averaging x-1 and x+1 gridpoints together
        dr[0] = r[1] / 2  # this doesn't matter as r=0 here
        dr[1] = (r[2] + r[1]) ** 2 / (8 * r[1])  # this line seems to be doing most of the job
        dr[N] = r[N] - r[N - 1]
        
        # The below would be more straightforward, but gives slightly worse results (the error is 20% bigger than Rob's, which gives a 0.1% bigger result for gnd_state than it should)
        # dr = r - np.roll(r, 1)
        # dr[0] = 0
        
        dr = dr.reshape(-1, 1).repeat(Nz, 1)
        r = r.reshape(-1, 1).repeat(Nz, 1)
        rho = rho.reshape(-1, 1).repeat(Nz, 1)  # reshaping arrays to be 2D
        R = b
        
        # Scaling factors for the (inverse) Hankel transform
        Hankel_scaling_1 = np.nan_to_num(R / J,posinf=0,neginf=0)
        Hankel_scaling_2 = J / beta
        
        inverse_Hankel_scaling_1 = np.nan_to_num(beta / J,posinf=0,neginf=0)
        inverse_Hankel_scaling_2 = J / R

    return rparams


def zsetup(Nz, N, gamma):
    """Defines the grid in the z direction."""

    class zparams:
        size_Z_gamma = 10 / math.sqrt(gamma)
        Zc = size_Z_gamma / 2
        z = np.linspace(-size_Z_gamma, 0, Nz)  # can use z = np.linspace(0, size_Z_gamma, Nz) to reverse grid - also need to reverse dzarray
        dz = size_Z_gamma / (Nz - 1)  # dz is the grid-resolution in the z-direction
        
        kz = rfftfreq(2 * Nz - 2, dz / (2 * np.pi))
        
        dzarray = np.full(Nz, dz)
        dzarray[0] = dz / 4  # this and the following two lines are to account for averaging along z (integrals)
        dzarray[1] = 3 * dz / 4
        dzarray[Nz - 1] = dz / 2
        
        dzarray = dzarray.reshape(1, -1).repeat(N + 1, 0)
        z = z.reshape(1, -1).repeat(N + 1, 0)  # will have shape N+1 x Nz
        kz = kz.reshape(1, -1).repeat(N + 1, 0)

    return zparams


def Udd_fun(kz, k_mag, Zc, rho):
    np.seterr(invalid='ignore')  # ignoring division by 0 as that is treated well with nan_to_num
    Udd = 3 * np.nan_to_num(kz / k_mag, posinf=0) ** 2 - 1
    next_term = 3 * np.cos(Zc * kz)
    next_term *= np.nan_to_num(2 * np.pi * rho / k_mag, posinf=0) ** 2
    next_term -= 3 * np.nan_to_num(2 * np.pi * rho / k_mag * kz / k_mag, posinf=0) * np.sin(Zc * kz)
    next_term *= np.exp(-Zc * 2 * np.pi * rho)
    Udd += next_term
    np.seterr(invalid='warn')
    return Udd  # this roughly follows eq. 2.6 (it seems 2pi*rho is somehow k_rho? not really sure what's going on)


def DHFT(psi, rparams, m):
    """Calculate the discrete Hankel-Fourier transform of psi, giving psi(rho, k_z)."""
    # Calculate the dct first
    psi_temp = create_psi_temp(psi)  # create temporary array for the Type-I DCT
    out = np.real(rfft_numpy(psi_temp, axis=1))  # calculate dct
    
    # Then calculate the Hankel transform
    out = piecewise_mult_1D(out, rparams.Hankel_scaling_1)  # changes variables
    out = np.matmul(rparams.C, out)  # applies transformation matrix
    out = piecewise_mult_1D(out, rparams.Hankel_scaling_2)
    
    return out


def IDHFT(psi, rparams, m):
    """Calculate the inverse discrete Hankel-Fourier transform of psi."""
    # Calculate the inverse Hankel transform first
    out = piecewise_mult_1D(psi, rparams.inverse_Hankel_scaling_1)  # changes variables
    out = np.matmul(rparams.C, out)  # applies transformation matrix
    out = piecewise_mult_1D(out, rparams.inverse_Hankel_scaling_2)
    
    # Then calculate the idct
    psi_temp = create_psi_temp(out)  # create temporary array for the Type-I DCT
    out = idct_normalize(rfft_numpy(psi_temp, axis=1))  # calculate idct
    
    return out


@njit(parallel=True, fastmath=True, nogil=True, cache=True)
def idct_normalize(a):
    """Normalising a Type-I DCT so that it becomes a Type-I IDCT. Returns a/(2*Nz-2)."""
    (N, Nz) = a.shape
    N -= 1
    out = np.empty((N + 1, Nz))
    norm = 2 * Nz - 2
    for i in prange(N + 1):
        for j in prange(Nz):
            out[i, j] = np.real(a[i, j]) / norm
    
    return out


@njit(parallel=True, fastmath=True, nogil=True, cache=True)
def create_psi_temp(a):
    """Create temporary psi array for DCT Type-I."""
    (N, Nz) = a.shape
    N -= 1
    Nz_real = 2 * Nz - 2
    out = np.empty((N + 1, Nz_real))
    for i in prange(N + 1):
        for j in prange(Nz):
            out[i, j] = a[i, j]
        for j in prange(Nz, Nz_real):
            out[i, j] = a[i, Nz_real - j]
    
    return out


@njit(parallel=True, fastmath=True, nogil=True, cache=True)
def piecewise_mult_2D(a, b):
    """Calculate Hadamard (piecewise) product of 2 arrays (2D a and 2D b)."""
    (N, Nz) = a.shape
    N -= 1
    out = np.empty((N + 1, Nz))
    for i in prange(N + 1):
        for j in prange(Nz):
            out[i, j] = a[i, j] * b[i, j]
    
    return out


@njit(parallel=True, fastmath=True, nogil=True, cache=True)
def piecewise_mult_1D(a, b):
    """Calculate Hadamard (piecewise) product of 2 arrays (2D a and 1D b)."""
    (N, Nz) = a.shape
    N -= 1
    out = np.empty((N + 1, Nz))
    for i in prange(N + 1):
        for j in prange(Nz):
            out[i, j] = a[i, j] * b[i, 0]
    
    return out


@njit(parallel=True, fastmath=True, nogil=True, cache=True)
def Norm_r(psi, dVol):
    """Calculating the normalisation factor for psi, taking the average of psi between gridpoints (see dzarray and notebook)."""
    (N, Nz) = psi.shape
    N -= 1
    integral = 0.0
    for i in prange(N + 1):
        for j in prange(Nz):
            integral += psi[i, j] ** 2 * dVol[i, j]
    
    return np.sqrt(integral)  # a normalised wavefunction should yield 1


def calc_dd_cont_cyl(psisq, rparams, Udd,m):
    out = DHFT(psisq, rparams,m)
    out = piecewise_mult_2D(out, Udd)
    out = IDHFT(out, rparams,m)
    return out


def energy(psi, V_trap, gs, D, rparams, dVol, k_mag, Udd,m):
    """Calculates the energy of a given psi."""
    psisq = psi ** 2
        
    KE_term = 0.5 * k_mag ** 2 * DHFT(psi, rparams,m)
    KE_term = psi * IDHFT(KE_term, rparams,m)
    
    integrand = KE_term
    integrand += V_trap * psisq
    integrand += gs / 2 * psisq ** 2
    dd_cont_cyl = D * calc_dd_cont_cyl(psisq, rparams, Udd,m)
    integrand += 0.5 * dd_cont_cyl * psisq
    
    return np.sum(integrand * dVol)


def Dcritguess(Nu, gamma, n):
    # out = np.exp(2.8 / n) * gamma ** (1 / 2 + 2 / n)
    out = (Nu * 1.5 * gamma ** ((6 + 3 * n / 2) / (3 * n + 4))) ** ((3 * n + 4) / (3 * n)) * 1.5 * np.pi ** (3 / 2) * math.gamma(1 + 2 / n) / math.gamma(5 / 2 + 2 / n) / (8 * np.pi)
    return out


@njit(parallel=True, fastmath=True, nogil=True, cache=True)
def apply_potential(dt, V_trap, gs, psisq, D, dd_cont, psi):
    """Applies half of the potential operator to psi. Returns np.exp(-0.5 * dt * (V_trap + gs * psisq + D * dd_cont)) * psi."""
    (N, Nz) = psi.shape
    N -= 1
    out = np.empty((N + 1, Nz))
    for i in prange(N + 1):
        for j in prange(Nz):
            out[i, j] = np.exp(-0.5 * dt * (V_trap[i, j] + gs * psisq[i, j] + D * dd_cont[i, j])) * psi[i, j]
    
    return out


def splitstep(psi, dt, V_trap, gs, D, rparams, kin, dVol, Udd,m):
    """Does a single split-operator imaginary-time iteration. We use dct to Fourier transform along z and then Hankel to Fourier transform along r. psi is assumed real."""
    psisq = psi ** 2
    dd_cont = calc_dd_cont_cyl(psisq, rparams, Udd,m)
    psi = apply_potential(dt, V_trap, gs, psisq, D, dd_cont, psi)  # applies first half of potential
    psi = piecewise_mult_2D(kin, DHFT(psi, rparams,m))  # applies kinetic operator in (cosine) Fourier and Bessel space
    psi = IDHFT(psi, rparams,m)  # transforms back to position space
    psisq = psi ** 2
    dd_cont = calc_dd_cont_cyl(psisq, rparams, Udd,m)
    psi = apply_potential(dt, V_trap, gs, psisq, D, dd_cont, psi)  # applies second half of potential
    psi = psi / Norm_r(psi, dVol)  # normalise the wavefunction
    return psi


def find_nu_crit(n, gamma):
    """Find the critical nu for a given n and gamma."""
    
    num_threads = 2  # os.cpu_count()
    init_threads(num_threads)
    
    N = 200
    Nz = 201
    m=0
    
    b = 1.2 * (50) ** (2 / (3 * n)) * gamma ** (1 / n)
    rparams = radialsetup(N, Nz, b)
    zparams = zsetup(Nz, N, gamma)
    
    """Setup checks"""
    
    if rparams.dr[2, 0] > 2 * np.pi / math.sqrt(gamma) / 5:
        print("Grid is too coarse to sample roton oscillations - finer grid required. Current dr =", rparams.dr[2, 0], ", maximum dr =", 2 * np.pi / math.sqrt(gamma) / 5)
        
    if Nz - 1 != next_fast_len(Nz - 1):
        print("Inefficient grid is being used, increase Nz to", next_fast_len(Nz - 1) + 1)
    
    """Final generic Definitions"""
    
    dVol = 4 * np.pi * zparams.dzarray * rparams.r * rparams.dr  # volume element for energy and norm
    k_mag = np.sqrt(zparams.kz ** 2 + (2 * np.pi * rparams.rho) ** 2)  # kmag is total magnitude of k vector
    # exp_T = np.exp((-1*dt*0.5*k_mag**2)) # for kinetic energy
    Udd = Udd_fun(zparams.kz, k_mag, zparams.Zc, rparams.rho)  # not including D
    Udd *= 4 * np.pi / 3  # this now has an extrac 4pi/3 factor added here
    
    """Define potential"""
    V_trap = 0.5 * ((rparams.r) ** n + (gamma * zparams.z) ** 2)
    
    """Non interacting ground states and initial psi"""
    if n < 6:
        gnd_state = np.exp((-rparams.r ** 2 - gamma * zparams.z ** 2) / 2)
        gnd_state /= (np.pi ** 3 / gamma) ** (1 / 4)
        # gnd_state /= Norm_r(gnd_state, dVol)
    else:
        gnd_state = np.exp(-gamma * zparams.z ** 2 / 2)
        gnd_state *= ss.jv(0, rparams.J0[0] * rparams.r) * (rparams.r < 1)
        gnd_state /= Norm_r(gnd_state, dVol)  # normalisation
    
    psi = gnd_state  # this by the way only copies the reference, not the underlying array itself
    psi /= Norm_r(psi, dVol)  # normalisation
    currentE = energy(psi, V_trap, 0, 0, rparams, dVol, k_mag, Udd)
    currentpsi = psi
    
    tic = time.perf_counter()
    
    """D loop"""
    
    dt0 = 1e-3
    Etol0 = 1e-4
    nu_crit = 0
    
    for iteration in range(3):
        if iteration == 0:
            print('Initial iteration 0')
            nulist = np.arange(0, 9, 1)
            Etol = Etol0
            
        elif iteration == 1:
            print('Start iteration 1')
            nu_crit_index = np.where(nulist == nu_crit)[0][0]
            print('Starting from nu = ', nulist[max(nu_crit_index - 2, 0)])
            psi = psilist[max(nu_crit_index - 2, 0)]
            print('Reloaded psi for restart')
            nulist = np.arange(np.max(nu_crit - 2, 0), nu_crit + 1, 0.25)
            # Etol /= 10
            
        elif iteration == 2:
            print('Start iteration 2')
            nu_crit_index = np.where(nulist == nu_crit)[0][0]
            print('Starting from nu = ', nulist[max(nu_crit_index - 4, 0)])
            psi = psilist[max(nu_crit_index - 4, 0)]
            print('Reloaded psi for restart')
            nulist = np.arange(np.max(nu_crit - 1, 0), nu_crit + 0.5, 0.05)
            Etol /= 100
            
        breaking_clause = 0
        Dlist = Dcritguess(nulist, gamma, n)
        gslist = np.zeros(nulist.size)
        # gslist=[0*4*np.pi*2/3*Dcritguess(5.6,gamma,n),0,0]
        psilist = np.zeros([nulist.size, N + 1, Nz])
        Elist = np.zeros(nulist.size)
        psidifflist = np.zeros(nulist.size)
        dpsiErrlist = np.zeros(nulist.size)
        bigpsidifflist = np.zeros(nulist.size)
        
        for (j, D) in enumerate(Dlist):
            print('gamma:', gamma, ', D:', D, ', nu:', nulist[j])
            dt = dt0
            gs = gslist[j]
            converged = False
            diverged = False
            stop = False
            
            p = 0
            while not stop:
                oldE = currentE
                oldpsi = currentpsi
                kin = np.exp(-1 * dt * 0.5 * k_mag ** 2)  # the kinetic energy operator
                i = 0
                p += 1
                converged = False
                
                while not converged and not diverged:
                    i += 1
                    Etemp = np.zeros(3)
                    psitemp = np.zeros((3, N + 1, Nz))
                    for _ in range(20):
                        psi = splitstep(psi, dt, V_trap, gs, D, rparams, kin, dVol, Udd)
                    Etemp[0] = energy(psi, V_trap, gs, D, rparams, dVol, k_mag, Udd)
                    psitemp[0] = psi
                    for _ in range(20):
                        psi = splitstep(psi, dt, V_trap, gs, D, rparams, kin, dVol, Udd)
                    Etemp[1] = energy(psi, V_trap, gs, D, rparams, dVol, k_mag, Udd)
                    psitemp[1] = psi
                    for _ in range(20):
                        psi = splitstep(psi, dt, V_trap, gs, D, rparams, kin, dVol, Udd)
                    Etemp[2] = energy(psi, V_trap, gs, D, rparams, dVol, k_mag, Udd)
                    psitemp[2] = psi
                    
                    Ediff = Etemp[2] - Etemp[1]
                    dErr = -Ediff / np.log((Etemp[1] - Etemp[0]) / Ediff)
                    currentE = Etemp[2]
                    print('currentE:', currentE, ', Ediff:', Ediff, ', dErr:', dErr, ', i:', i, ', dt:', dt)
                    
                    psidiff1 = np.average((psitemp[1] - psitemp[0]) ** 2)
                    psidiff2 = np.average((psitemp[2] - psitemp[1]) ** 2)
                    dpsiErr = -psidiff2 / np.log(psidiff1 / psidiff2)
                    currentpsi = psitemp[2]
                    print('psidiff1:', psidiff1, ', psidiff2:', psidiff2, ', dpsiErr:', dpsiErr, 'nu :', nulist[j])
                    
                    if ((np.abs(dErr) < Etol / 3 and dErr * Ediff <= 0 and np.abs(Ediff) < Etol / 10) or (Ediff == 0 and np.isnan(dErr))) and i > 5:
                        converged = True
                        print("iteration converged")
                    elif np.isnan(Ediff):
                        converged = False
                        diverged = True
                        breaking_clause = 1
                        print(breaking_clause, "Ediff is NaN - restarting")
                    elif Etemp[2] < 0:
                        converged = False
                        diverged = True
                        breaking_clause = 2
                        print(breaking_clause, "Energy is negative - restarting")
                    elif np.abs(Ediff) > 5 and i > 50:
                        converged = False
                        diverged = True
                        breaking_clause = 3
                        print(breaking_clause, "Ediff is large after many iterations, probably just going around in circles - restarting")
                    """elif (dErr * Ediff >= 0 or dpsiErr * psidiff2 >= 0) and i > 50:
                        converged = False
                        diverged = True
                        stop = True
                        breaking_clause = 3
                        print(breaking_clause, "Started diverging - restarting")"""
                
                print('currentE:', currentE, ', currentE-oldE:', currentE - oldE, '(delta_psi)^2:', np.average((currentpsi - oldpsi) ** 2), ', p:', p)
                if converged and np.abs(currentE - oldE) < Etol / 2 and p > 2:
                    print("nu = ", nulist[j], " converged")
                    print('saving psi for nu = ', nulist[j])
                    psilist[j] = psi
                    Elist[j] = currentE
                    psidifflist[j] = psidiff1
                    dpsiErrlist[j] = dpsiErr
                    bigpsidifflist[j] = np.average((currentpsi - oldpsi) ** 2)
                    stop = True
                elif diverged:
                    print("nu = ", nulist[j], " diverged")
                    nu_crit = nulist[j]
                    stop = True
                else:
                    dt /= 1.26
                    print('nu =', nulist[j], 'didn\'t coverge/diverge so far - decreasing dt')
            
            if D == 0:
                print('Calculating gamma_real')
                psi_gnd = psilist[0]
                sigma_r = np.sqrt(np.sum(rparams.r ** 2 * psi_gnd ** 2 * dVol))
                sigma_z = np.sqrt(np.sum(zparams.z ** 2 * psi_gnd ** 2 * dVol))
                gamma_real = sigma_r / sigma_z
                
            if diverged:
                break
            
        print('psidiff1: %.1e' % psidifflist[j - 1], ', dpsiErr: %.1e' % dpsiErrlist[j - 1], '(delta_psi)^2: %.1e' % bigpsidifflist[j - 1])    
        if iteration == 0:
            print('Iteration 0 over - restarting')
        elif iteration == 1:
            print('Iteration 1 over - restarting')
        elif iteration == 2:
            print('Iteration 2 over - finished')
        
    toc = time.perf_counter()
    print('time of run: %d' % (toc - tic))
    
    np.savez_compressed('psiarrayfor_n=' + str(n) + 'and_gamma=' + str(gamma) + '.npz', psilist)
    
    n_r = np.sum(psilist ** 2 * 2 * zparams.dzarray, axis=2)
    np.savez_compressed('n_rarrayfor_n=' + str(n) + 'and_gamma=' + str(gamma) + '.npz', n_r)
    np.savez_compressed('rarrayfor_n=' + str(n) + 'and_gamma=' + str(gamma) + '.npz', rparams.r)
    
    n_z = np.sum(psilist ** 2 * 2 * np.pi * rparams.r * rparams.dr, axis=1)
    np.savez_compressed('n_zarrayfor_n=' + str(n) + 'and_gamma=' + str(gamma) + '.npz', n_z)
    np.savez_compressed('zarrayfor_n=' + str(n) + 'and_gamma=' + str(gamma) + '.npz', zparams.z)    
    
    exportlist = np.empty([nulist.size, 7])
    exportlist[:, 0] = nulist
    exportlist[:, 1] = Dlist
    exportlist[:, 2] = gslist
    exportlist[:, 3] = Elist
    exportlist[:, 4] = psidifflist
    exportlist[:, 5] = dpsiErrlist
    exportlist[:, 6] = bigpsidifflist
    np.savetxt('DandElistfor_n=' + str(n) + 'and_gamma=' + str(gamma) + '.dat', exportlist, header='nu, D, gs, E, psidiff, dpsiErr, bigpsidiff')
    
    stablist = np.empty(9)
    stablist[0] = gamma
    stablist[1] = gamma_real
    stablist[2] = Dlist[j - 1]
    stablist[3] = nulist[j - 1]
    stablist[4] = breaking_clause
    stablist[5] = dt0
    stablist[6] = Etol
    stablist[7] = mkl.get_max_threads()
    stablist[8] = numba.get_num_threads()
    print('Etol:', Etol, 'dt0:', dt0)
    print(stablist)
    return stablist


"""**************** main program *****************"""

# if __name__ == '__main__':
    
#     """For use with the cluster, we could supply the list of parameters to run in a file paramlist.txt."""
#     if len(sys.argv) == 2:
#         lineno = int(sys.argv[1])
#         try:
#             params = np.loadtxt('paramlist.txt', skiprows=lineno, max_rows=1)
#             n = params[0]
#             gamma = params[1]
#             print('n, gamma:', n, gamma)
#         except OSError:
#             print('paramlist.txt not found, going ahead with hardcoded values')
    
#     gammalist = np.linspace(10, 1000, 100)
#     gammalist = [1000]
    
#     n = 6
#     nlist = np.full_like(gammalist, n)
    
#     stablist = np.zeros([np.size(gammalist), 9])
    
#     tic = time.perf_counter()
    
    
#     # iterate over the nu list for each gamma in parallel
#     with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
#         results = executor.map(find_nu_crit, nlist, gammalist)
#         for (i, result) in enumerate(results):
#             stablist[i] = result
    
#     # uncomment the following two lines of code for serial execution (and comment the lines above)
#     # for (i, gamma) in enumerate(gammalist):
#     #    stablist[i] = find_nu_crit(n, gamma)
    
#     toc = time.perf_counter()
#     print(toc - tic)
    
#     np.savetxt('extrastabfor_n=' + str(n) + '.dat', stablist, header='gamma_program, gamma_real, D_crit, nu_crit, breaking_clause, dt0, Etol, MKL threads, Numba threads')
#     plt.plot(1, 1)
#     plt.show()


'**************** main program *****************'

In [15]:
#'''Importing data''''
#'C://Users/david/OneDrive/Documents/UNI/BEC project/diagrams/box potential stability plots/Plots with fixed gamma box case/'+fo


psis_D0_to_D40 = np.load('C://Users/david/OneDrive/Documents/UNI/BEC project/diagrams/harmonic stability plots/Plots with fixed gamma harmonic case/plots for gamma=7 using HFCT 2021-04-05/list of final psis,gamma = 7.npy')
psis_D0_to_D40.shape

(101, 101, 41)

In [123]:
psis_D30_to_D35[:,:,5]
Ds_30_to_35[-1]

35.0

In [3]:

""""""""""""""""""""""
parameters needed for BdG equations 
"""""""""""""""""""""

n=2
N = 100
Nz = 101
gamma = 7
b = 1.2 * (50) ** (2 / (3 * n)) * gamma ** (1 / n)
zparams = zsetup(Nz, N, gamma)
gs=0

In [5]:
m_array=np.array((0,))#np.array((0,1,2,3,4,5))
D_values = np.array((0,))#np.linspace(0,40,41)
BdG_energies = np.zeros((len(m_array),len(D_values)))
BdG_f_excitations = np.zeros((N+1,Nz,len(m_array),len(D_values))) #3d array to add psis to



for i in tqdm_notebook(range(len(m_array)),leave=False):
    m = m_array[i]
    rparams = radialsetup(N, Nz, b,m)
    gnd_state = np.exp((-rparams.r ** 2 - gamma * zparams.z ** 2) / 2)
    gnd_state /= (np.pi ** 3 / gamma) ** (1 / 4)

    dVol=4*np.pi*zparams.dzarray*rparams.r*rparams.dr ## Volume element for Energy and Norm
    k_mag = np.sqrt(zparams.kz**2 + (2*np.pi*rparams.rho)**2) ## kmag is total magnitude of k vector.
    V_trap = 0.5*((rparams.r)**n + (gamma*zparams.z)**2)
    Udd = Udd_fun(zparams.kz, k_mag, zparams.Zc, rparams.rho)  # not including D
    Udd *= 4 * np.pi / 3  # this now has an extrac 4pi/3 factor added here
    for j in tqdm_notebook(range(len(D_values)),leave=False):
        psi = psis_D0_to_D40[:,:,j]
        psi = gnd_state
        psi = interpolation(psi,rparams,m)
        D = D_values[j]
        mu = energy(psi, V_trap, gs, D, rparams, dVol, k_mag, Udd,m)
        evals, evecs = BdG_lowest_excitation_energy_order_m(N,Nz,D,Udd,psi,m,mu,interp=False)
        BdG_energies[i,j] = np.min(evals)#[0]
        BdG_f_excitations[:,:,i,j] = np.reshape(evecs[:,np.argmin(evals)],(N+1,Nz))
    



BdG_energies = np.transpose(BdG_energies)
plt.plot(D_values,BdG_energies)

plt.legend(np.append('m=0',['m = '+str(s) for s in m_array[1:]]))
plt.grid()
#plt.ylabel(r'$\math{\\\boldsymbol{\\\psi}(r,0)}$',fontsize=13)
#if self.save_boolean:
#    plt.savefig(self.path+'/'+'psi in z=0 plane, gamma = '+str(self.gamma)+'.png')
plt.show() ##plots psi at z=0 for various D values 

C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide
C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


KeyboardInterrupt: 

In [28]:
psi = psis_D0_to_D40[:,:,0]
evals, evecs = BdG_lowest_excitation_energy_order_m(N,Nz,D,Udd,psi,m,mu,interp=True)


C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide
C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


ArpackNoConvergence: ARPACK error -1: No convergence (5001 iterations, 0/3 eigenvectors converged) [ARPACK error -14: SNAUPD did not find any eigenvalues to sufficient accuracy.]

In [3]:
import mpl_toolkits.mplot3d as mplot3d
evecs_small = np.reshape(evecs_small,(N+1,Nz))
# psi = evecs_small
plt.plot(rparams.r,evecs_small[:,-1])
plt.show()
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(rparams.r, zparams.z,np.abs(evecs_small))
plt.show()

NameError: name 'evecs_small' is not defined

In [63]:
print(psi_10[:,0])
print(psi[:,0])

[1.92874985e-22 1.86634297e-22 1.80877643e-22 1.75029896e-22
 1.68682599e-22 1.61845803e-22 1.54579832e-22 1.46960679e-22
 1.39070327e-22 1.30992393e-22 1.22809474e-22 1.14601225e-22
 1.06442853e-22 9.84039230e-23 9.05474303e-23 8.29291169e-23
 7.55970221e-23 6.85912514e-23 6.19439492e-23 5.56794545e-23
 4.98146185e-23 4.43592578e-23 3.93167177e-23 3.46845188e-23
 3.04550599e-23 2.66163532e-23 2.31527672e-23 2.00457577e-23
 1.72745696e-23 1.48168933e-23 1.26494662e-23 1.07486114e-23
 9.09070601e-24 7.65258114e-24 6.41184976e-24 5.34716744e-24
 4.43842895e-24 3.66690613e-24 3.01533340e-24 2.46794740e-24
 2.01048794e-24 1.63016683e-24 1.31561141e-24 1.05678881e-24
 8.44916464e-25 6.72363808e-25 5.32549361e-25 4.19836691e-25
 3.29432083e-25 2.57286128e-25 2.00000842e-25 1.54743441e-25
 1.19167451e-25 9.13414740e-26 6.96856492e-26 5.29155911e-26
 3.99934545e-26 3.00856327e-26 2.25265324e-26 1.67878276e-26
 1.24525879e-26 9.19368658e-27 6.75592363e-27 4.94133988e-27
 3.59724221e-27 2.606511